# Eat Safe, Love

## Notebook Set Up

In [73]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [74]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [75]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [76]:
# review the collections in our database
# Get the list of collection names in the database
collection_names = db.list_collection_names()

# Print the collection names
for collection_name in collection_names:
    print(collection_name)

establishments


In [77]:
# assign the collection to a variable
establishments = db['establishments']


## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [78]:


# Define the query to find establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Find the establishments with a hygiene score of 20
results = collection.find(query)

# Count the number of documents in the result
num_documents = collection.count_documents(query)
print("Number of establishments with a hygiene score of 20:", num_documents)

# Check if there are documents in the result
if num_documents > 0:
    # Display the first document in the results
    first_document = next(results)
    pprint(first_document)

    # Convert the results to a DataFrame
    df = pd.DataFrame(results)

    # Print the number of rows in the DataFrame
    print("Number of rows in the DataFrame:", len(df))

    # Display the first 10 rows
    print("First 10 rows of the DataFrame:")
    print(df.head(10))
else:
    print("No establishments found with a hygiene score of 20.")


Number of establishments with a hygiene score of 20: 41
{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': '0',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64ac489b4851275aaa062536'),
 'geocode': {'latitude': '50.769705', 'longitude': '0.27694'},
 'latitude': None,
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/110681',


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [80]:

import pandas as pd

# Query to filter establishments in City of London Corporation with RatingValue >= 4
query = {
    
    "RatingValue": {"$gte": 4},
    "LocalAuthorityName": "City of London Corporation"
}

# Find the establishments matching the query
results = collection.find(query)

# Count the number of documents in the result
num_establishments = collection.count_documents(query)
print("Number of establishments in City of London Corporation with RatingValue >= 4:", num_establishments)

# Display the first document in the results using pprint
if num_establishments > 0:
    first_document = next(results)
    pprint(first_document)

# Convert the results to a DataFrame
df = pd.DataFrame(results)

# Print the number of rows in the DataFrame
num_rows = len(df)
print("Number of rows in the DataFrame:", num_rows)

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))



Number of establishments in City of London Corporation with RatingValue >= 4: 0
Number of rows in the DataFrame: 0
First 10 rows of the DataFrame:
Empty DataFrame
Columns: []
Index: []


In [ ]:
# Convert the result to a Pandas DataFrame

# Display the number of rows in the DataFrame

# Display the first 10 rows of the DataFrame


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 
longitude = 

query = 
sort =  

# Print the results


In [ ]:
# Convert result to Pandas DataFrame


In [87]:

from pymongo import MongoClient, GEOSPHERE

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017")
database = client["uk_food"]
collection = database["establishments"]

# Query documents with missing or invalid geocode values
query = {
    "$or": [
        {"geocode.longitude": {"$type": "string"}},
        {"geocode.latitude": {"$type": "string"}}
    ]
}
results = collection.find(query)

# Update documents with correct geocode values
for result in results:
    longitude = result["geocode"]["longitude"]
    latitude = result["geocode"]["latitude"]
    result["geocode"]["longitude"] = float(longitude)
    result["geocode"]["latitude"] = float(latitude)
    collection.replace_one({"_id": result["_id"]}, result)

# Create a geospatial index on the geocode field
collection.create_index([("geocode", GEOSPHERE)])



'geocode_2dsphere'

In [88]:
# Set the latitude and longitude of Penang Flavours
latitude = 51.490142
longitude = 0.083840

# Define the degree search range
degree_search = 0.01

# Create the query
query = {
    "RatingValue": 5,
    "geocode": {
        "$nearSphere": {
            "$geometry": {
                "type": "Point",
                "coordinates": [longitude, latitude]
            },
            "$maxDistance": degree_search / 111.12  # Convert search range to meters
        }
    }
}

# Sort the results by hygiene score in ascending order
sort = [("scores.Hygiene", 1)]

# Execute the query
results = collection.find(query).sort(sort).limit(5)

# Print the results
for result in results:
    pprint(result)

# Convert the result to a Pandas DataFrame
df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print("Number of rows in the DataFrame:", len(df))

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))


Number of rows in the DataFrame: 0
First 10 rows of the DataFrame:
Empty DataFrame
Columns: []
Index: []


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

# Print the number of documents in the result

# Print the first 10 results


In [ ]:
# Convert the result to a Pandas DataFrame

# Display the number of rows in the DataFrame

# Display the first 10 rows of the DataFrame


In [90]:
from pymongo import MongoClient
import pandas as pd

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017")
database = client["uk_food"]
collection = database["establishments"]

# Create the pipeline
pipeline = [
    {"$match": {"scores.Hygiene": 0}},
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}}
]

# Execute the pipeline
result = list(collection.aggregate(pipeline))

# Count the number of documents in the result
count = len(result)

print(f"Total establishments with a hygiene score of 0: {count}")

# Print the first 10 results
for doc in result[:10]:
    print(f"Local Authority: {doc['_id']}, Count: {doc['count']}")

# Convert the result to a Pandas DataFrame
df = pd.DataFrame(result)

# Display the number of rows in the DataFrame
print(f"Number of rows in DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
print(df.head(10))


Total establishments with a hygiene score of 0: 55
Local Authority: Thanet, Count: 1130
Local Authority: Greenwich, Count: 882
Local Authority: Maidstone, Count: 713
Local Authority: Newham, Count: 711
Local Authority: Swale, Count: 686
Local Authority: Chelmsford, Count: 680
Local Authority: Medway, Count: 672
Local Authority: Bexley, Count: 607
Local Authority: Southend-On-Sea, Count: 586
Local Authority: Tendring, Count: 542
Number of rows in DataFrame: 55
               _id  count
0           Thanet   1130
1        Greenwich    882
2        Maidstone    713
3           Newham    711
4            Swale    686
5       Chelmsford    680
6           Medway    672
7           Bexley    607
8  Southend-On-Sea    586
9         Tendring    542
